<h1>Recordlinkage: Matching Country Names <span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-Description" data-toc-modified-id="Problem-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Description</a></span></li><li><span><a href="#Purpose" data-toc-modified-id="Purpose-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Purpose</a></span></li><li><span><a href="#Import-&amp;-Prepare-Data" data-toc-modified-id="Import-&amp;-Prepare-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import &amp; Prepare Data</a></span><ul class="toc-item"><li><span><a href="#Clean-&amp;-Reshape" data-toc-modified-id="Clean-&amp;-Reshape-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Clean &amp; Reshape</a></span></li></ul></li><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Record Linkage</a></span><ul class="toc-item"><li><span><a href="#Exact-Matches" data-toc-modified-id="Exact-Matches-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Exact Matches</a></span></li><li><span><a href="#Tfidf-and-Cosine-Similarity" data-toc-modified-id="Tfidf-and-Cosine-Similarity-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Tfidf and Cosine-Similarity</a></span><ul class="toc-item"><li><span><a href="#See-also" data-toc-modified-id="See-also-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>See also</a></span><ul class="toc-item"><li><span><a href="#LaTeX-Notes" data-toc-modified-id="LaTeX-Notes-4.2.1.1"><span class="toc-item-num">4.2.1.1&nbsp;&nbsp;</span>LaTeX Notes</a></span></li></ul></li><li><span><a href="#Which-are-the-features-extracted-from-the-country-names?" data-toc-modified-id="Which-are-the-features-extracted-from-the-country-names?-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Which are the features extracted from the country names?</a></span></li><li><span><a href="#Tfidf-Matrix" data-toc-modified-id="Tfidf-Matrix-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Tfidf Matrix</a></span></li><li><span><a href="#Cosine-Similarity-in-loops" data-toc-modified-id="Cosine-Similarity-in-loops-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Cosine Similarity in loops</a></span><ul class="toc-item"><li><span><a href="#Take-a-look-at-the-results,-pay-attention-to:" data-toc-modified-id="Take-a-look-at-the-results,-pay-attention-to:-4.2.4.1"><span class="toc-item-num">4.2.4.1&nbsp;&nbsp;</span>Take a look at the results, pay attention to:</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Combine-Exact--and-Cosine-Similarity-Matches" data-toc-modified-id="Combine-Exact--and-Cosine-Similarity-Matches-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Combine Exact  and Cosine-Similarity Matches</a></span><ul class="toc-item"><li><span><a href="#Alternative-Cosine-Similarity-Results-Display" data-toc-modified-id="Alternative-Cosine-Similarity-Results-Display-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Alternative Cosine-Similarity Results Display</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li></ul></div>

In [3]:
import time as tm
import pandas as pd
start_time = tm.strftime("%m/%d/%Y, %A, %H:%M%p")

datatime documentation: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

In [5]:
print("This jupyter notebook was created on", str(start_time))

This jupyter notebook was created on 05/21/2023, Sunday, 12:13PM


# Problem Description

Multiple sources provide official or unofficial world country lists (including territories/regiones with undetermined/disputed sovereignty). It is helpful for data users to have their choice of country list linked to different versions of country list.

Different versions of country list can make data merging by exact matches more dificcult. While the names of most countries are consistent, some are not. For example, the Department of State uses the term "SAINT KITTS AND NEVIS	", whereas the country was spelled as "St. Kitts-Nevis" in the 2017-2021 U.S. Census ACS Data Dictionary. Moreover, within the U.S. Census Bureau, more than one country lists co-exist. Except for the country list that is used to determine a person's country of origin (e.g., ACS) in survey data, there is also the *Schedule C - Country Codes and Descriptions*. Both Census country lists provide associated country codes.

Additionally, spelling or spacing variations exist within the same data source. For example, both "HONG KONG S. A. R." and "HONG KONG S.A.R." were presented from the same source. Minor differences like this would add more work to generate basic descriptive statistics.

# Purpose

This jupyter notebook documents my attempts to link slighty different country names. Specifically, I try to link an unmatched country name to a country code from a different data source.


# Import & Prepare Data

1. Department of State Visa Statistics: country names seeking matches of country codes
2. U.S. Census country list (Schedule C): reference/dictionary data that have both country names and codes

In [17]:
# read in data
nonimm = pd.read_csv("nonimm.csv")
# The following is added after confirming with the full country name of 
#"The United Kingdom of Great ritain and Northern Ireland"
import warnings
warnings.filterwarnings("ignore")
nonimm['nationality'][nonimm['nationality'].str.contains("GREAT BRITAIN")] = "UNITED KINGDOM"
# extract unique country names
nationality = list(set(nonimm['nationality'])) # use list() to wrap it
print("`nationality` from the Visa Statistics data have", str(len(nationality)),"countries.")
# create country frequency table
country_freq =nonimm['nationality'].value_counts().to_frame().\
reset_index().rename(columns = {'nationality':'freq','index':'country'}).\
sort_values(by = ['country']).reset_index(drop = True)

`nationality` from the Visa Statistics data have 232 countries.


## Clean & Reshape

- Read in `.txt` rows with observations
- Reshape data

In [28]:
# skip non-observation rows upon data import
ScheduleC_raw = pd.read_csv("country.txt", delimiter = "\t",\
                            skiprows = [0, 1, 2, 3, 4, 245, 246, 247, 248], names = ['V'])

In [29]:
print(ScheduleC_raw.head(2))

                                                   V
0  1000     |    United States of America        ...
1  1010     |    Greenland                       ...


In [30]:
# reshape data by spliting the column by |
ScheduleC = ScheduleC_raw['V'].str.split('|', expand = True).\
rename(columns = {0:'code', 1:'name', 2:'iso'})
# remove potential leading and trailing spaces in a dataframe
ScheduleC = ScheduleC.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# delete the raw data since the cleaned & reshaped dataframe is ready
del ScheduleC_raw
# set to uppercase to be consistent with the other dataset
ScheduleC['name'] = ScheduleC['name'].str.upper()

In [31]:
ScheduleC.head()

,code,name,iso
0,1000,UNITED STATES OF AMERICA,US
1,1010,GREENLAND,GL
2,1220,CANADA,CA
3,1610,SAINT PIERRE AND MIQUELON,PM
4,2010,MEXICO,MX


# Record Linkage

## Exact Matches

In [32]:
exact_matches = country_freq[['country']].\
merge(ScheduleC, left_on = 'country',right_on = 'name', how = 'inner').\
drop('name', axis = 1)
print("Between the two data sources,",str(exact_matches.shape[0]),\
      "country names were exactly the same. They can be joined directly.",\
      "\nThe first few rows of the joined dataset:")
exact_matches.head(3)

Between the two data sources, 186 country names were exactly the same. They can be joined directly. 
The first few rows of the joined dataset:


,country,code,iso
0,AFGHANISTAN,5310,AF
1,ALBANIA,4810,AL
2,ALGERIA,7210,DZ


In [33]:
country_freq_short = \
country_freq[['country']][~country_freq['country'].isin(exact_matches['country'])].reset_index(drop = True)
print(str(country_freq_short.shape[0]), "countries didn't find exact matches in the country code dataframe.",\
     "Here're the first few rows of the unmatched countries:")
country_freq_short.head()

46 countries didn't find exact matches in the country code dataframe. Here're the first few rows of the unmatched countries:


,country
0,**ESWATINI
1,*NON-NATIONALITY BASED ISSUANCES
2,*NON-NATIONLITY BASED ISSUANCES
3,"BAHAMAS, THE"
4,BOSNIA-HERZEGOVINA


The above unmatched records will be handled using the following techniques:

## Tfidf and Cosine-Similarity

$$similarity(A,B) = cos(\theta) = \frac{A \times B}{\|A\| \|B\|} = \frac{\sum\limits_{i = 1}^{n} A_{i} B_{i}}{\sqrt{\sum\limits_{i = 1}^{n} A^{2}_{i}} \sqrt{\sum\limits_{i = 1}^{n} B^{2}_{i}}}$$

where

- $\theta$ is the angle between the vectors,
- $A \times B$ is dot product between $A$ and $B$ and calculated as $A \times B = A^{T}B = \Sigma^{n}_{i = 1} A_{i}B_{i} = A_{1}B_{1} + A_{2}B_{2} + ... + A_{n}B_{n}$
- $\|A\|$ represents the $L_{2}$ norm or magnitude of the vector which is calculated as $\|A\| = \sqrt{A^{2}_{1} + A^{2}_{2} + ... + A^{2}_{n}}$

### See also
- What is Cosine Similarity? https://www.learndatasci.com/glossary/cosine-similarity/
- Cosine similarity: How does it measure the similarity, Maths behind and usage in Python https://towardsdatascience.com/cosine-similarity-how-does-it-measure-the-similarity-maths-behind-and-usage-in-python-50ad30aad7db

#### LaTeX Notes
- This is the LaTeX code to generate the formula:
    >`similarity(A,B) = cos(\theta) = \frac{A \times B}{\|A\| \|B\|} = \frac{\sum\limits_{i = 1}^{n} A_{i} B_{i}}{\sqrt{\sum\limits_{i = 1}^{n} A^{2}_{i}} \sqrt{\sum\limits_{i = 1}^{n} B^{2}_{i}}}`

- The Comprehensive LateX Symbol List, http://tug.ctan.org/info/symbols/comprehensive/symbols-a4.pdf

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
# for "richer" text data, it is recommended to add the `min_df = ` statement
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')
# fit the `tfidf_vectorizer` with the country names with country codes
corpus = tfidf_vectorizer.fit_transform(ScheduleC['name'])
print("The type of `corpus` is", str(type(corpus)) + '.',\
     "\nThe following steps show how to view the contents in the corpus.")

The type of `corpus` is <class 'scipy.sparse._csr.csr_matrix'>. 
The following steps show how to view the contents in the corpus.


### Which are the features extracted from the country names?

- `features` are the tokenized words from all country names

In [35]:
# depending on package versions, the following syntax may vary slightly
print("The country names have been converted to",\
      str(len(tfidf_vectorizer.get_feature_names_out())),\
      "features (tokenized words). \nStopwords such as 'the' and 'of' have been removed. \n")
print("First 20 features:",tfidf_vectorizer.get_feature_names_out()[:20],\
      "\n...\nand last 20 features:\n... \n",tfidf_vectorizer.get_feature_names_out()[-20:])

The country names have been converted to 298 features (tokenized words). 
Stopwords such as 'the' and 'of' have been removed. 

First 20 features: ['administered' 'afghanistan' 'africa' 'african' 'albania' 'algeria'
 'america' 'american' 'andorra' 'angola' 'anguilla' 'antarctic' 'antigua'
 'arab' 'arabia' 'argentina' 'armenia' 'aruba' 'australia' 'austria'] 
...
and last 20 features:
... 
 ['uganda' 'ukraine' 'united' 'uruguay' 'uzbekistan' 'vanuatu' 'vatican'
 'venezuela' 'verde' 'vietnam' 'vincent' 'virgin' 'wallis' 'west'
 'western' 'yemen' 'za' 'zambia' 'zealand' 'zimbabwe']


### Tfidf Matrix

In [36]:
# Create a data frame of the tfidf matrix, column names are the features
tfidf_matrix_df = pd.DataFrame(corpus.toarray(),\
                               columns = tfidf_vectorizer.get_feature_names_out())

### Cosine Similarity in loops

1. Loop over the unmatched rows that seek for potential matches
2. Convert (tokenize and then vectorize) the individual rows into a query vector
3. Keep the top N best matches at our discretion
    - by setting a cut-off cosine-similarity score; and
    - by selecting one or multiple potential matches, and
    - finding a match is not guaranteed
4. Append matching results in a dataframe using `.copy` and `pd.concat()`

In [37]:
%%time
from sklearn.metrics.pairwise import cosine_similarity
# Write a loop of i over the items in the unmatched observations that seek for a matching country name and code
# This is NOT the labeled/coded/dictionary (however we call it) dataframe as shown in the tfidf matrix
n = 2 # Keep top 2 best matches
for i in range(len(country_freq_short['country'])):
    # the ith observation in the unmatched data as individual query
    query = country_freq_short['country'].iloc[i]
    # tokenize and vectorize each individual query row, must be wrap with []
    query_vector = tfidf_vectorizer.transform([query])
    # run cosine-similarity, corpus is the fitted and transformed(vectorized) ScheduleC dataframe
    cosine_sim = pd.DataFrame(cosine_similarity(corpus, query_vector),\
                             columns = ['cosine_similarity'],\
                             index = ScheduleC.index)
    cosine_sim = cosine_sim.sort_values(by = ['cosine_similarity'], ascending = False)
    # Keep top n best matches, and keep all columns(`,:]`) in the ScheduleC (coded/labeled/dictionary)
    # output gives potential matches as country names
    output = ScheduleC.loc[cosine_sim.index[0:n],:]
    output.index = ['match_1','match_2']
    # scores report the cosine similarity score of these matches
    scores = cosine_sim[0:n]
    
    # NOW, CREATE FINAL OUTPUTS
    if i == 0:
        # `deep = True`, a new object will be created with a copy of the calling object’s data and indices
        all_outputs = output.copy(deep = True) # for the first iteration, just copy the result
        all_scores = scores.copy(deep = True)
    else:
        # from the 2nd iteration onwards
        all_outputs = pd.concat([all_outputs,output]) # append new rows as iterations continue
        all_scores = pd.concat([all_scores, scores])

CPU times: user 93.6 ms, sys: 3.51 ms, total: 97.1 ms
Wall time: 95.2 ms


#### Take a look at the results, pay attention to:

- Where did the indeces appear? They repeat over each query.
- Which are the rows and the columns? These are the same columns in the ScheduleC dataframe.
- What else can be done to make the final results more interpretable?
    1. Convert the indeces to a regular column
    2. Join the potential matches (country names & codes) with the cosine similarity score
    3. Join the original query to the final results. This way, we can see which country has been matched to which countries in a different dataframe.

In [38]:
# Convert the indeces to a regular column
all_outputs['rank'] = all_outputs.index
# Must reset index for all dataframes before merging
all_outputs = all_outputs.reset_index(drop = True)

In [39]:
# Must reset index for all dataframes before merging
all_scores = all_scores.reset_index(drop = True)

In [40]:
import numpy as np
# duplicate query rows to join the queries to their results
all_matches = pd.concat([pd.DataFrame(np.repeat(country_freq_short[['country']].values, n, axis = 0)),\
                        all_outputs, all_scores], axis = 1) # query, potential matches, cosine-similarity score

In [41]:
print("From the merged final results, we can see that more work can be done to improve the output.",
     "\n 1.Rename the query column whose current column name is 0.",
     "\n 2.Remove rows with `cosine_similarity = 0`",
     "\n 3.Set cut-off cosine-similarity score"
     "\n 4.Optional: reorder the columns\n")
all_matches

From the merged final results, we can see that more work can be done to improve the output. 
 1.Rename the query column whose current column name is 0. 
 2.Remove rows with `cosine_similarity = 0` 
 3.Set cut-off cosine-similarity score
 4.Optional: reorder the columns



,0,code,name,iso,rank,cosine_similarity
0,**ESWATINI,7950,ESWATINI,SZ,match_1,1.000000
1,**ESWATINI,1000,UNITED STATES OF AMERICA,US,match_2,0.000000
2,*NON-NATIONALITY BASED ISSUANCES,1000,UNITED STATES OF AMERICA,US,match_1,0.000000
3,*NON-NATIONALITY BASED ISSUANCES,1010,GREENLAND,GL,match_2,0.000000
4,*NON-NATIONLITY BASED ISSUANCES,1000,UNITED STATES OF AMERICA,US,match_1,0.000000
...,...,...,...,...,...,...
87,VATICAN CITY,1000,UNITED STATES OF AMERICA,US,match_2,0.000000
88,WESTERN SAHARA,6150,SAMOA (WESTERN SAMOA),WS,match_1,0.473538
89,WESTERN SAHARA,1000,UNITED STATES OF AMERICA,US,match_2,0.000000
90,YEMEN,5210,YEMEN (REPUBLIC OF YEMEN),YE,match_1,0.949060


In [42]:
# More dataframe editions/enhancements
# After observing the results, I decided to set the cutoff at 0.5
all_matches = all_matches[all_matches['cosine_similarity'] > 0.5]
all_matches = all_matches.rename(columns = {0:'query', 'name':'match'})
all_matches = all_matches[['query','match','code','iso','cosine_similarity','rank']]
all_matches = all_matches.reset_index(drop = True)

In [43]:
print("After some filter work, the following shows the reasonable matches.",\
      "\nMost remain rows are the top match (rank == 'match_1'), with a few 'match_2' presented.\n",\
     str(len(set(all_matches['query']))),"out of",str(len(set(country_freq_short['country']))),\
     "previously unmatched countries have found at least one match.\n")
from IPython.display import display, HTML
display(HTML(all_matches.to_html()))

After some filter work, the following shows the reasonable matches. 
Most remain rows are the top match (rank == 'match_1'), with a few 'match_2' presented.
 37 out of 46 previously unmatched countries have found at least one match.



,query,match,code,iso,cosine_similarity,rank
0,**ESWATINI,ESWATINI,7950,SZ,1.000000,match_1
1,"BAHAMAS, THE",BAHAMAS,2360,BS,1.000000,match_1
2,BOSNIA-HERZEGOVINA,BOSNIA AND HERZEGOVINA,4793,BA,1.000000,match_1
3,BRITISH NATIONAL OVERSEAS (HONG KONG) PASSPORT,HONG KONG,5820,HK,0.835529,match_1
4,BURMA,BURMA (MYANMAR),5460,MM,0.707107,match_1
5,CHINA - MAINLAND,CHINA,5700,CN,1.000000,match_1
6,CHINA - TAIWAN,TAIWAN,5830,TW,0.707107,match_1
7,CHINA - TAIWAN,CHINA,5700,CN,0.707107,match_2
8,CHINA-MAINLAND,CHINA,5700,CN,1.000000,match_1
9,CHINA-TAIWAN,TAIWAN,5830,TW,0.707107,match_1


In [44]:
print("At this point, I don't see any `match_2` should be favored over `match_1`.",\
     "Now, I can remove all `match_2` rows.")
best_match = all_matches[all_matches['rank'] == "match_1"]
best_match = best_match[['query','code','iso']].rename(columns = {'query':'country'})
best_match.columns

At this point, I don't see any `match_2` should be favored over `match_1`. Now, I can remove all `match_2` rows.


Index(['country', 'code', 'iso'], dtype='object')

In [45]:
print("Join the best match dataframe from cosine-similarity to the exact matches.")
final_matches = pd.concat([exact_matches, best_match]).sort_values(by = 'country').reset_index(drop = True)

Join the best match dataframe from cosine-similarity to the exact matches.


# Combine Exact  and Cosine-Similarity Matches

In [52]:
print("Now, take a look at the full matched data:\n")
from IPython.display import display, HTML
display(HTML(final_matches[:20].to_html()))

Now, take a look at the full matched data:



,country,code,iso
0,**ESWATINI,7950,SZ
1,AFGHANISTAN,5310,AF
2,ALBANIA,4810,AL
3,ALGERIA,7210,DZ
4,ANDORRA,4271,AD
5,ANGOLA,7620,AO
6,ANGUILLA,2481,AI
7,ANTIGUA AND BARBUDA,2484,AG
8,ARGENTINA,3570,AR
9,ARMENIA,4631,AM


In [47]:
unmatched = list(set(nationality) - set(final_matches['country']))
print("These countries/regions from the Department of State Visa Statistics dataframe couldn't find matches:\n"\
      ,unmatched,"\nTOTAL COUNT:", str(len(unmatched)),\
      "\nWorking on them manually is doable.\n")
unused_ScheduleC = list(set(ScheduleC['name']) - set(all_matches['match']) - set(exact_matches['country']))
print("The following are the unmatched countries/regions in the ScheduleC data:\n",unused_ScheduleC,\
     "\nTOTAL COUNT:", str(len(unused_ScheduleC)))

These countries/regions from the Department of State Visa Statistics dataframe couldn't find matches:
 ['NON-NATIONALITY BASED ISSUANCES', 'MACAU S. A. R.', 'SWAZILAND', '*NON-NATIONLITY BASED ISSUANCES', 'PALESTINIAN AUTHORITY TRAVEL DOCUMENT', 'LAOS', '*NON-NATIONALITY BASED ISSUANCES', 'WESTERN SAHARA', 'MACAU S.A.R.'] 
TOTAL COUNT: 9 
Working on them manually is doable.

The following are the unmatched countries/regions in the ScheduleC data:
 ['NORFOLK ISLAND', 'HEARD ISLAND AND MCDONALD ISLANDS', 'COOK ISLANDS', 'MACAO', 'MARTINIQUE', 'PITCAIRN ISLANDS', 'BRITISH INDIAN OCEAN TERRITORY', 'CHRISTMAS ISLAND (IN THE INDIAN OCEAN)', 'VIRGIN ISLANDS OF THE UNITED STATES', 'UNITED STATES OF AMERICA', 'WEST BANK ADMINISTERED BY ISRAEL', 'NEW CALEDONIA', 'SVALBARD AND JAN MAYEN', 'FALKLAND ISLANDS (ISLAS MALVINAS)', 'FRENCH SOUTHERN AND ANTARCTIC LANDS', 'SINT MAARTEN', 'NORTHERN MARIANA ISLANDS', 'TOKELAU', 'FRENCH POLYNESIA', 'GREENLAND', 'REUNION', 'GUAM', 'NIUE', 'PUERTO RICO', 'COCO

## Alternative Cosine-Similarity Results Display
- This display style may be preferred when multiple matches are possible
- Compared to the previous work, this alternative is a transposed view of the results.

In [48]:
k = 3
for i in range(len(country_freq_short)):
    query = country_freq_short['country'].iloc[i]
    query_vector = tfidf_vectorizer.transform([query])
    cosine_sim = pd.DataFrame(cosine_similarity(corpus, query_vector),\
                             columns = ['cosine_similarity'],\
                             index = ScheduleC.index)
    cosine_sim = cosine_sim.sort_values(by = ['cosine_similarity'], ascending = False)
    output = ScheduleC.loc[cosine_sim.index[0:k],:]
    output.index = ['match_1','match_2','match_3']
    # Here's the difference in codes:
    result_transposed = pd.DataFrame(output['name']).transpose()
    if i == 0:
        all_transposed = result_transposed.copy(deep = True)
    else:
        all_transposed = pd.concat([all_transposed, result_transposed])

In [49]:
all_transposed = all_transposed.reset_index(drop = True)
all_transposed.shape

(46, 3)

In [50]:
pd.concat([country_freq_short[['country']], all_transposed], axis = 1)

,country,match_1,match_2,match_3
0,**ESWATINI,ESWATINI,UNITED STATES OF AMERICA,KIRIBATI
1,*NON-NATIONALITY BASED ISSUANCES,UNITED STATES OF AMERICA,GREENLAND,COCOS (KEELING) ISLANDS
2,*NON-NATIONLITY BASED ISSUANCES,UNITED STATES OF AMERICA,GREENLAND,COCOS (KEELING) ISLANDS
3,"BAHAMAS, THE",BAHAMAS,UNITED STATES OF AMERICA,TUVALU
4,BOSNIA-HERZEGOVINA,BOSNIA AND HERZEGOVINA,UNITED STATES OF AMERICA,NORFOLK ISLAND
5,BRITISH NATIONAL OVERSEAS (HONG KONG) PASSPORT,HONG KONG,BRITISH VIRGIN ISLANDS,BRITISH INDIAN OCEAN TERRITORY
6,BURMA,BURMA (MYANMAR),UNITED STATES OF AMERICA,TUVALU
7,CHINA - MAINLAND,CHINA,UNITED STATES OF AMERICA,NORFOLK ISLAND
8,CHINA - TAIWAN,TAIWAN,CHINA,UNITED STATES OF AMERICA
9,CHINA-MAINLAND,CHINA,UNITED STATES OF AMERICA,NORFOLK ISLAND


limit_output extension: Maximum message size of 10000 exceeded with 10060 characters

# References

1. 2017-2021 ACS PUMS Data Dictionary, https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2017-2021.pdf
2. U.S. Department of State Visa Statistics, https://travel.state.gov/content/travel/en/legal/visa-law0/visa-statistics.html
3. Schedule C (Code of Federal Regulations, Title 15, Subtitle B) - Country List (by code), Foreigh Trade Division, U.S. Census Bureau, https://www.census.gov/foreign-trade/schedules/c/country.txt

In [53]:
end_time = tm.strftime("%m/%d/%Y, %A, %H:%M%p")
print("This jupyter notebook was last updated on", str(end_time))

This jupyter notebook was last updated on 05/28/2023, Sunday, 23:16PM
